# Le classifieur de Bayes ☕️

## Introduction

Reformalisons notre problème de classification. Soit $\mathcal{X}$ l'espace des données d'entrée et $\mathcal{Y}$ celui des données de sorties. On aurait par exemple $\mathcal{X}$ l'ensemble de toutes les photos et $\mathcal{Y}$ celui des labels "chien" et "chat". A priori, dans le cadre d'une application de détection "chien/chat", on ne s'attend pas à trouver n'importe quelle image (e.g. on ne s'attend pas à voir des photos de pizzas). L'idée est d'interpréter cela de manière probabiliste en considérons le couple de variables aléatoires suivant : 

$$X, Y\in\mathcal{X}\times\mathcal{Y}.$$

On appellera $\mu$ la mesure de $X$ dans le sens où étant donné $A\subseteq\mathcal{X}$, on a $\mathbb{P}(X\in A)=\mu(A)$ et $\eta(x)=\mathbb{P}(Y=1|X=x)$ (dans le cas de la classification binaire) la "probabilité a posteriori". La fonction $\eta$ revête d'autres formes lorsqu'on travaille sur d'autres problèmes (e.g. classification, top-k). Il est important de constater que $\mu$ et $\eta$ décrivent totalement les deux variables aléatoires $X, Y$. En effet, soit $A\subseteq \mathcal{X}\times \mathcal{Y}$, nous avons :

$$\mathbb{P}(X, Y\in A)=\int_{A\cap \mathcal{X}\times \{1\}}\eta(x)d\mu+\int_{A\cap \mathcal{X}\times \{0\}}(1-\eta(x))d\mu.$$

Notre objectif est de construire une application $h:\mathcal{X}\mapsto\mathcal{Y}$ telle que les "prédictions" de $h$ soient bonnes dans le sens du risque suivant :

$$L(h)=\mathbb{P}(h(X)\neq Y)=\mathbb{E}\big[\textbf{1}\{h(X)\neq Y\}\big].$$

On veut que la probabilité que $h$ fasse des erreurs soient la plus faible possible.

C'est parque ne connaissant ni $\mu$ ni $\eta$ que nous ne pouvons pas calculer $L$ et donc trouver le meilleur $h$. Habituellement, en *machine learning*, nous collectons des données afin d'estimer $L$ et construire le classifieur approprié.

## Le classifieur de Bayes

Supposons que nous connaissions $\eta$ (nous sommes un oracle). Quel est le meilleur classifieur que nous puissions construire ?

Il s'agit du classifieur suivant :

$$g^\star(x)=\begin{cases}1&\text{ si }\eta(x)\geq 0.5\\ 0&\text{ sinon.}\end{cases}$$

C'est ce qu'on appelle le classifieur de Bayes. C'est lui qui fait le moins d'erreurs (notons qu'il peut exister plusieurs classifieurs qui font aussi peu d'erreurs). On peut quantifier le risque atteint par ce classifieur :

$$L^\star=\mathbb{E}\big[\text{min}(\eta(X), 1-\eta(X))\big].$$

Si les labels sont déterministes (i.e. $\eta\in\{0, 1\}$), alors $L^\star=0$.

----
**Proposition.**

$\not\exists g:\mathcal{X}\mapsto\mathcal{Y}$ tel que $L(g)<L^\star$. Dit autrement, on ne peut pas faire mieux que le classifieur de Bayes.

**Preuve.**

Soit $x\in\mathcal{X}$ et $g:\mathcal{X}\mapsto\mathcal{Y}$ un classifieur quelconque. On a :

$$\begin{aligned}
\mathbb{P}(g(X)\neq Y|X=x)&=1-\mathbb{P}(g(X)=Y|X=x)\\
&= 1-(\mathbb{P}(Y=1, g(X)=1|X=x)+\mathbb{P}(Y=0, g(X)=1|X=x))\\
&=1-(\textbf{1}\{g(x)=1\}\mathbb{P}(Y=1|X=x)+\textbf{1}\{g(x)=0\}\mathbb{P}(Y=0|X=x))\\
&=1-(\textbf{1}\{g(x)=1\}\eta(x)+\textbf{1}\{g(x)=0\}(1-\eta(x))).
\end{aligned}$$

Ainsi, nous avons :

$$\begin{aligned}
\mathbb{P}(g(X)\neq Y|X=x)-\mathbb{P}(g^\star(X)\neq Y|X=x)&=\eta(x)(\textbf{1}\{g(x)=1\}-\textbf{1}\{g^\star(x)=1\})+(1-\eta(x))(\textbf{1}\{g(x)=0\}-\textbf{1}\{g^\star(x)=0\})\\
&=(2\eta(x)-1)(\textbf{1}\{g(x)=1\}-\textbf{1}\{g^\star(x)=1\})\\
&\geq 0
\end{aligned}$$

Il suffit maintenant de prendre l'espérance sur tous les "x" et le résultat est là.

---

## Un petit exercice

Construisons un jeu de données de classification binaire.

In [ ]:
import numpy as np
from scipy.special import expit as sigmoid

beta = np.random.uniform(-10, 10, size=(2, 1))

def class_probability(x):
    logit = np.dot(x, beta)
    proba = sigmoid(logit)
    
    return proba

def dataset(n):
    X = np.random.uniform(-1, 1, size=(n, 2))
    y = np.random.binomial(n=1, p=class_probability(X))
    return X, y

X, y = dataset(50)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

---
<span style="color:blue">**Exercice :**</span> **En vous appuyant sur le code précédent (en réutilisant éventuellement du code fourni), implémentez le classifieur de Bayes**

---

In [ ]:
def bayes_classifier(x):
    ####### Complete this part ######## or die ####################
    return (class_probability(x)>=0.5).astype(int)
    ###############################################################

In [ ]:
plt.figure(figsize=(12, 8))
XX, YY = np.mgrid[-1:1:500j, -1:1:500j]
predictions = bayes_classifier(np.stack([XX, YY], axis=2).reshape(500*500, 2))

plt.pcolormesh(XX, YY, predictions.reshape(500, 500), cmap=plt.cm.Paired, shading='auto')
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.title('Le classifieur de Bayes')
plt.show()